In [1]:
from leukemia_extract import extract_data
from api_extraction import api_data_extraction
from api_transformations import process_world_bank_data
from merge import merge_dataframes
from dimensional_model_transform import process_dimensions
from gx_validations import validation_results
from load_data import export_to_postgres

In [2]:
leukemia_df = extract_data()

/home/v4lentin4/Leukemia-Cancer-Risk-ETL/airflow/functions/leukemia_extract.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=conn.connection)


In [3]:
api_df = api_data_extraction()

File /home/v4lentin4/Leukemia-Cancer-Risk-ETL/data/world_bank_data.csv already exists. Loading data from the file.


In [ ]:
api_df = process_world_bank_data(api_df)

,country,co2_emissions_per_capita,nuclear_energy_pct,agri_employment_pct,fertilizer_consumption,gdp_per_capita,pm25_pollution,undernourishment_rate,alcohol_consumption_liters
0,argentina,4.035680,5.484569,0.612884,255.653331,14187.482725,14.624551,3.4,7.69
1,australia,14.014672,0.000000,2.147565,252.374500,64820.911789,7.387603,2.5,10.57
2,brazil,2.271017,2.049616,8.202819,391.647128,10294.866681,14.927492,3.7,8.36
3,canada,14.340252,14.301098,1.271544,24.823229,53431.185706,7.795220,2.5,9.92
4,china,9.399266,4.667542,22.333710,98.189967,12614.061742,50.299193,2.5,5.26


In [5]:
df = merge_dataframes(leukemia_df, api_df)
df.head()

,id,age,gender,country,wbc_count,rbc_count,platelet_count,hemoglobin_level,bone_marrow_blasts,genetic_mutation,...,leukemia_status,living_status,co2_emissions_per_capita,nuclear_energy_pct,agri_employment_pct,fertilizer_consumption,gdp_per_capita,pm25_pollution,undernourishment_rate,alcohol_consumption_liters
0,1,52,Male,china,2698,5.36,262493,12.2,72,1,...,0,1,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
1,2,15,Female,china,4857,4.81,277877,11.9,97,1,...,1,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
2,12,3,Male,china,6973,5.60,295077,11.9,19,0,...,0,1,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
3,56,60,Male,china,8233,5.50,194008,16.2,79,0,...,1,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
4,62,78,Male,china,7685,5.09,354128,13.0,81,1,...,0,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26


In [6]:
df = process_dimensions(df)

In [7]:
validation_results = validation_results(df)
print(validation_results)

Calculating Metrics: 100%|██████████| 74/74 [00:00<00:00, 449.01it/s]


True


In [8]:
export_to_postgres(df, validation_results)

Successfully connected to PostgreSQL database
Successfully exported Fact_Leukemia (143076 records)
Successfully exported Dim_Region (22 records)
Successfully exported Dim_PatientInfo (143076 records)
Successfully exported Dim_MedicalHistory (255 records)
Database connection closed
